# PyTorch Basic

PyTorch는 머신 러닝 라이브러리로, 주로 딥 러닝 알고리즘을 구현하고 실험하기 위해 사용됩니다. PyTorch는 Python 기반으로, 다양한 머신 러닝 및 딥 러닝 알고리즘의 개발과 연구를 용이하게 하기 위해 설계되었습니다. 이 라이브러리는 쉽게 사용하고 확장할 수 있는 구조를 제공하여 연구자와 개발자 모두에게 유용합니다.

[PyTorch Homepage](https://pytorch.org/)

![PyTorch Logo](https://github.com/jphan32/CVTrack-Tutorials/blob/main/resources/pytorch.png?raw=true)

### PyTorch의 주요 특징
#### 동적 계산 그래프(Dynamic Computational Graph)
- PyTorch는 동적 계산 그래프를 사용하여, 연산을 진행하는 동안 그래프를 구축합니다. 이는 개발자에게 높은 유연성을 제공하며, 디버깅도 용이합니다.

#### 자동 미분(Automatic Differentiation)
- PyTorch의 autograd 시스템은 연산 그래프를 통해 자동 미분을 수행하여 그래디언트를 계산합니다. 이는 복잡한 네트워크에서도 역전파를 쉽게 구현할 수 있게 해줍니다.

#### 다양한 라이브러리와 도구
- PyTorch는 torchvision, torchaudio, torchtext 등 다양한 도메인 특화 라이브러리를 제공하여, 다양한 애플리케이션에 적용할 수 있습니다.

#### 커뮤니티와 지원
- PyTorch는 강력한 커뮤니티 지원을 받고 있으며, 다양한 튜토리얼과 예제를 통해 빠르게 학습하고 응용할 수 있습니다.

#### 확장성
- PyTorch는 쉽게 사용자 정의 연산을 추가하고, GPU 가속을 활용할 수 있어, 확장성이 높습니다.
---

## 간단한 선형 회귀 모델 구현해보기
- 목표 : 캘리포니아 주택 가격 예측
- 데이터 : Colab 샘플 데이터

In [ ]:
#@title 라이브러리 Import

import pandas as pd
import numpy as np

import seaborn as sns
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from scipy import stats
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

from tqdm import tqdm

In [ ]:
#@title 데이터 불러오기 & EDA

# CSV 파일에서 데이터 불러오기
df_csv = pd.read_csv('sample_data/california_housing_train.csv')

# 결측치 제거
df_csv = df_csv.dropna(inplace=True)

# Z-score를 이용한 이상치 제거
z_scores = stats.zscore(df_csv[['median_income', 'median_house_value']])
abs_z_scores = np.abs(z_scores)
filtered_entries = (abs_z_scores < 3).all(axis=1)
df_csv = df_csv[filtered_entries]

# 상관 분석
correlation_matrix = df_csv.corr()
sns.heatmap(data=correlation_matrix, annot=True)
plt.show()

# 산점도 그래프
sns.scatterplot(data=df_csv, x='median_income', y='median_house_value')
plt.show()

In [ ]:
#@title 데이터 정의

# 데이터 정규화: MinMaxScaler를 사용하여 스케일링
scaler = MinMaxScaler()
df_csv[['median_income', 'median_house_value']] = scaler.fit_transform(df_csv[['median_income', 'median_house_value']])

# 특성과 레이블 선택
x = df_csv[['median_income']].values
y = df_csv[['median_house_value']].values

# 데이터 분할
x_train, x_val, y_train, y_val = train_test_split(x, y, test_size=0.3, random_state=42)

# 텐서 변환
x_train_tensor = torch.tensor(x_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
x_val_tensor = torch.tensor(x_val, dtype=torch.float32)
y_val_tensor = torch.tensor(y_val, dtype=torch.float32)

# 데이터셋과 데이터 로더 준비
train_dataset = TensorDataset(x_train_tensor, y_train_tensor)
val_dataset = TensorDataset(x_val_tensor, y_val_tensor)

batch_size = 2000
train_loader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
val_loader = DataLoader(dataset=val_dataset, batch_size=batch_size)

In [ ]:
#@title 모델 정의

class LinearRegression(nn.Module):
    def __init__(self):
        super(LinearRegression, self).__init__()
        # 선형 계층 정의 (입력 차원, 출력 차원)
        self.linear = nn.Linear(1, 1)  # median_income은 1차원, median_house_value도 1차원

    def forward(self, x):
        return self.linear(x)

model = LinearRegression()

In [ ]:
#@title 손실 함수와 옵티마이저 정의

criterion = nn.MSELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [ ]:
#@title 선형회귀 모델 학습

# epoch 설정
num_epochs = 1000

# Early Stopping을 위한 변수 초기화
early_stopping_patience = 5
early_stopping_counter = 0
best_loss = float('inf')

# 학습
for epoch in range(num_epochs):
    model.train()
    total_loss = 0  # 에폭에 대한 총 손실 초기화

    for batch_idx, (features, targets) in enumerate(train_loader):
        # 순전파
        outputs = model(features)
        loss = criterion(outputs, targets)

        # 역전파
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()  # 배치의 손실 누적

    # 에포크당 평균 손실 계산 및 출력
    avg_train_loss = total_loss / len(train_loader)

    # 검증 데이터셋에 대한 평가
    model.eval()
    total_val_loss = 0
    with torch.no_grad():
        for features, targets in val_loader:
            predictions = model(features)
            val_loss = criterion(predictions, targets)
            total_val_loss += val_loss.item()

    # 에포크당 평균 검증 손실 계산
    avg_val_loss = total_val_loss / len(val_loader)

    # Early Stopping 체크
    print(f'Epoch [{epoch+1}/{num_epochs}], Train Loss: {avg_train_loss:.6f}, Validation Loss: {avg_val_loss:.6f}')
    if avg_val_loss < best_loss:
        best_loss = avg_val_loss
        early_stopping_counter = 0  # 최고 기록 갱신 시 카운터 초기화
    else:
        early_stopping_counter += 1  # 기록 갱신 못할 시 카운터 증가

    if early_stopping_counter >= early_stopping_patience:
        print(f'Early stopping triggered after epoch {epoch+1}')
        break

## 테스트 데이터셋 평가 및 결과 시각화
- 목표 : 캘리포니아 주택 가격 예측
- 데이터 : Colab 샘플 데이터

In [ ]:
# 테스트 데이터셋 불러오기
df_test_csv = pd.read_csv('sample_data/california_housing_test.csv')

# 결측치가 있다면 제거
df_test_csv.dropna(inplace=True)

# 데이터 정규화: MinMaxScaler를 사용하여 스케일링
# 학습 데이터에 대해 이미 fit이 수행된 scaler를 사용해야 하므로,
# 테스트 데이터에는 fit_transform 대신 transform을 적용합니다.
# 가정: 'scaler'는 학습 데이터에 대해 이미 fit 되었습니다.
df_test_csv[['median_income', 'median_house_value']] = scaler.transform(df_test_csv[['median_income', 'median_house_value']])

# 텐서 변환
x = df_csv[['median_income']].values
y = df_csv[['median_house_value']].values
x_test_tensor = torch.tensor(x, dtype=torch.float32)
y_test_tensor = torch.tensor(y, dtype=torch.float32)

# 데이터셋과 데이터 로더 준비
test_dataset = TensorDataset(x_test_tensor, y_test_tensor)
batch_size = 2000
test_loader = DataLoader(dataset=test_dataset, batch_size=batch_size)


# 테스트 데이터셋에 대한 평가
model.eval()

# 예측값을 저장할 리스트 초기화
predictions, actuals = [], []

# 테스트 데이터에 대한 예측 수행
with torch.no_grad():
    for X_batch, y_batch in test_loader:
        y_test_pred = model(X_batch)
        predictions.append(y_test_pred.numpy())
        actuals.append(y_batch.numpy())